In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
from video_processor import VideoProcessor
from tensorflow.keras.preprocessing.sequence import pad_sequences
import requests

# Archive 1

In [2]:
def _convert_result_to_string(result):
    output = '['
    for i, r in enumerate(result):
        if i==20: break
        output += r.tojson() + ','
    output = output[:-1]+']'
    return eval(output), r.orig_shape

In [3]:
def _extract_keypoints(data, orig_shape):
  d={}

  keypoints_batch =[]
  batch_size=20
  max_persons = 10
  for i, frame in enumerate(data):
    track_id_keypoints = []
    for person_id in range(max_persons):
      
      if person_id>=len(frame):
        track_id_keypoints = d.get(person_id, [])
        track_id_keypoints.append(np.zeros(34))
        d[person_id] = track_id_keypoints

      else:
          keypoints = frame[person_id]["keypoints"]
          x = np.array(keypoints["x"]) / orig_shape[1]
          y = np.array(keypoints["y"]) / orig_shape[0]
          XY = np.asarray([j for j in zip(x, y)]).reshape(34)
      
          track_id_keypoints = d.get(person_id, [])
          track_id_keypoints.append(XY)
          d[person_id] = track_id_keypoints

  return np.asarray(list(d.values()))



In [4]:
# image_path = 'examples/videos/V12.mp4'
# result = model.track(image_path, device='0', show=True, verbose=False, stream=True)
# cv2.destroyAllWindows()

# data, orig_shape = _convert_result_to_string(result)

# keypoints_batch = _extract_keypoints(data, orig_shape)

# keypoints_batch.shape
# cv2.destroyAllWindows()

In [5]:
# load model
from keras.models import load_model
fusion_model = load_model('models/detector_combined.keras')
fusion_model.input_shape

(None, 10, 34)

In [6]:
def predictor(data):
    prediction = fusion_model.predict(data, verbose=False).tolist()
    # classes = ['boxing', 'handwaving', 'jogging', 'running', 'walking', 'handclapping']
    # p = []
    # for i in prediction:
    #     p.append([classes[i.index(max(i))], max(i)])
    # return p
    return fusion_model.predict(data, verbose=False)

In [11]:
# Violence test
import time
model = YOLO('models/yolov8n-pose.pt') 
cap = cv2.VideoCapture("examples/videos/Das_Wunder_von_Bern_(Deutschland_2002_2003)_kick_ball_f_cm_np1_fr_goo_2.avi")

buffer_size = fusion_model.input_shape[1]
keypoints_buffer = np.zeros((10,1,34))

vp = VideoProcessor(model, max_frames=1, img_sz=320, show_stream=False)

while True:
    ret, frame = cap.read()
    # img_resp = requests.get(url) 
    # img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8) 
    # img = cv2.imdecode(img_arr, -1) 
    
    if ret == False:
        cv2.destroyAllWindows()
        cap.release()
        print("Error reading frame!")
        break

    height, width, channels = frame.shape

    if keypoints_buffer.shape[1] != buffer_size:
        keypoints_batch = vp.process_image(frame)
        keypoints_buffer = np.concatenate((keypoints_buffer, keypoints_batch), axis=1)

        # result = model.track(frame, persist=True, verbose=False, device='0')
        # data, orig_shape = _convert_result_to_string(result)
        # keypoints_batch = _extract_keypoints(data, orig_shape)
        
        # print(keypoints_buffer.shape)

    else:

        
        keypoints_batch = vp.process_image(frame)


        # result = model.track(frame, persist=True, verbose=False, device='0')
        # data, orig_shape = _convert_result_to_string(result)
        # keypoints_batch = _extract_keypoints(data, orig_shape)
        
        if keypoints_buffer.shape[1] + keypoints_batch.shape[1] > buffer_size:
            excess_size = keypoints_buffer.shape[1] + keypoints_batch.shape[1] - buffer_size
            keypoints_buffer = keypoints_buffer[:, excess_size:, :]

        # Concatenate the new keypoints_batch to the circular buffer
        keypoints_buffer = np.concatenate((keypoints_buffer, keypoints_batch), axis=1)
        start_time = time.time()
        
        prediction = predictor(keypoints_buffer)
        
        end_time = time.time()  # Stop the timer after the loop
        execution_time = end_time - start_time
        print("Execution time:", execution_time, "seconds")

        for id, person in enumerate(vp.data[0]):
            x1 = int(person['box']['x1'])
            y1 = int(person['box']['y1'])
            x2 = int(person['box']['x2'])
            y2 = int(person['box']['y2'])
            # print((prediction[id][1]))
            # print([(x1,y1), (x2,y2), prediction[id]], end='\r')

            color = (0,0,255) if prediction[id][0] > 0.7 else (0,255,0)             
            frame = cv2.rectangle(frame, (x1,y1), (x2,y2), color, 5)
            # frame = cv2.putText(frame, prediction[id][0],(x1,y1),fontFace=1, fontScale=2, color=color, thickness= 1)
        

    cv2.imshow('Output',frame)

    if cv2.waitKey(1)==27:
        cv2.destroyAllWindows()
        cap.release()
        break

 
    
  

Execution time: 0.08961296081542969 seconds
Execution time: 0.09252047538757324 seconds
Execution time: 0.0930325984954834 seconds
Execution time: 0.10413455963134766 seconds
Execution time: 0.09251856803894043 seconds
Execution time: 0.0955345630645752 seconds
Execution time: 0.09652447700500488 seconds
Execution time: 0.08450651168823242 seconds
Execution time: 0.08659791946411133 seconds
Execution time: 0.08352017402648926 seconds
Execution time: 0.08457541465759277 seconds
Execution time: 0.08351874351501465 seconds
Execution time: 0.08581137657165527 seconds
Error reading frame!


In [22]:
# Violence test
import time
model = YOLO('models/yolov8n-pose.pt') 
cap = cv2.VideoCapture("examples/videos/likebeckam_walk_f_cm_np1_ri_med_31.avi")

buffer_size = fusion_model.input_shape[1]
keypoints_buffer = np.zeros((10,1,34))

vp = VideoProcessor(model, max_frames=1, img_sz=320, show_stream=False)

while True:
    ret, frame = cap.read()
    # img_resp = requests.get(url) 
    # img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8) 
    # img = cv2.imdecode(img_arr, -1) 
    
    if ret == False:
        cv2.destroyAllWindows()
        cap.release()
        print("Error reading frame!")
        break

    height, width, channels = frame.shape

    if keypoints_buffer.shape[1] != buffer_size:
        keypoints_batch = vp.process_image(frame)
        keypoints_buffer = np.concatenate((keypoints_buffer, keypoints_batch), axis=1)

        # result = model.track(frame, persist=True, verbose=False, device='0')
        # data, orig_shape = _convert_result_to_string(result)
        # keypoints_batch = _extract_keypoints(data, orig_shape)
        
        # print(keypoints_buffer.shape)

    else:

        
        keypoints_batch = vp.process_image(frame)


        # result = model.track(frame, persist=True, verbose=False, device='0')
        # data, orig_shape = _convert_result_to_string(result)
        # keypoints_batch = _extract_keypoints(data, orig_shape)
        
        if keypoints_buffer.shape[1] + keypoints_batch.shape[1] > buffer_size:
            excess_size = keypoints_buffer.shape[1] + keypoints_batch.shape[1] - buffer_size
            keypoints_buffer = keypoints_buffer[:, excess_size:, :]

        # Concatenate the new keypoints_batch to the circular buffer
        keypoints_buffer = np.concatenate((keypoints_buffer, keypoints_batch), axis=1)
        start_time = time.time()
        
        prediction = predictor(keypoints_buffer)
        
        end_time = time.time()  # Stop the timer after the loop
        execution_time = end_time - start_time
        print("Execution time:", execution_time, "seconds")

        for id, person in enumerate(vp.data[0]):
            x1 = int(person['box']['x1'])
            y1 = int(person['box']['y1'])
            x2 = int(person['box']['x2'])
            y2 = int(person['box']['y2'])
            # print((prediction[id][1]))
            # print([(x1,y1), (x2,y2), prediction[id]], end='\r')

            color = (0,0,255) if prediction[id][0] > 0.7 else (0,255,0)             
            frame = cv2.rectangle(frame, (x1,y1), (x2,y2), color, 5)
            # frame = cv2.putText(frame, prediction[id][0],(x1,y1),fontFace=1, fontScale=2, color=color, thickness= 1)
        

    cv2.imshow('Output',frame)

    if cv2.waitKey(1)==27:
        cv2.destroyAllWindows()
        cap.release()
        break

 
    
  

Execution time: 0.08454632759094238 seconds
Execution time: 0.0835261344909668 seconds
Execution time: 0.08609628677368164 seconds
Execution time: 0.08455085754394531 seconds
Execution time: 0.08324599266052246 seconds
Execution time: 0.08452200889587402 seconds
Execution time: 0.08753395080566406 seconds
Execution time: 0.08809304237365723 seconds
Execution time: 0.08658027648925781 seconds
Execution time: 0.09451603889465332 seconds
Execution time: 0.09000730514526367 seconds
Execution time: 0.08558130264282227 seconds
Execution time: 0.08640766143798828 seconds
Execution time: 0.0825202465057373 seconds
Execution time: 0.08658027648925781 seconds
Execution time: 0.08552122116088867 seconds
Execution time: 0.0905153751373291 seconds
Execution time: 0.08603072166442871 seconds
Execution time: 0.08352184295654297 seconds
Execution time: 0.08369565010070801 seconds
Execution time: 0.08055257797241211 seconds
Execution time: 0.08954215049743652 seconds
Execution time: 0.07956433296203613

In [20]:
# Non Violence sample
import time
model = YOLO('models/yolov8n-pose.pt') 
cap = cv2.VideoCapture("examples/videos/v_WalkingWithDog_g18_c04.avi")

buffer_size = fusion_model.input_shape[1]
keypoints_buffer = np.zeros((10,1,34))

vp = VideoProcessor(model, max_frames=1, img_sz=320, show_stream=False)

while True:
    ret, frame = cap.read()
    # img_resp = requests.get(url) 
    # img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8) 
    # img = cv2.imdecode(img_arr, -1) 
    
    if ret == False:
        cv2.destroyAllWindows()
        cap.release()
        print("Error reading frame!")
        break

    height, width, channels = frame.shape

    if keypoints_buffer.shape[1] != buffer_size:
        keypoints_batch = vp.process_image(frame)
        keypoints_buffer = np.concatenate((keypoints_buffer, keypoints_batch), axis=1)

        # result = model.track(frame, persist=True, verbose=False, device='0')
        # data, orig_shape = _convert_result_to_string(result)
        # keypoints_batch = _extract_keypoints(data, orig_shape)
        
        # print(keypoints_buffer.shape)

    else:

        
        keypoints_batch = vp.process_image(frame)


        # result = model.track(frame, persist=True, verbose=False, device='0')
        # data, orig_shape = _convert_result_to_string(result)
        # keypoints_batch = _extract_keypoints(data, orig_shape)
        
        if keypoints_buffer.shape[1] + keypoints_batch.shape[1] > buffer_size:
            excess_size = keypoints_buffer.shape[1] + keypoints_batch.shape[1] - buffer_size
            keypoints_buffer = keypoints_buffer[:, excess_size:, :]

        # Concatenate the new keypoints_batch to the circular buffer
        keypoints_buffer = np.concatenate((keypoints_buffer, keypoints_batch), axis=1)
        start_time = time.time()
        
        prediction = predictor(keypoints_buffer)
        
        end_time = time.time()  # Stop the timer after the loop
        execution_time = end_time - start_time
        print("Execution time:", execution_time, "seconds")

        for id, person in enumerate(vp.data[0]):
            x1 = int(person['box']['x1'])
            y1 = int(person['box']['y1'])
            x2 = int(person['box']['x2'])
            y2 = int(person['box']['y2'])
            # print((prediction[id][1]))
            # print([(x1,y1), (x2,y2), prediction[id]], end='\r')

            color = (0,0,255) if prediction[id][0] > 0.7 else (0,255,0)             
            frame = cv2.rectangle(frame, (x1,y1), (x2,y2), color, 5)
            # frame = cv2.putText(frame, prediction[id][0],(x1,y1),fontFace=1, fontScale=2, color=color, thickness= 1)
        

    cv2.imshow('Output',frame)

    if cv2.waitKey(1)==27:
        cv2.destroyAllWindows()
        cap.release()
        break

 
    
  

Execution time: 0.08224272727966309 seconds
Execution time: 0.08563041687011719 seconds
Execution time: 0.0875554084777832 seconds
Execution time: 0.08800411224365234 seconds
Execution time: 0.09051346778869629 seconds
Execution time: 0.08654332160949707 seconds
Execution time: 0.08651876449584961 seconds
Execution time: 0.08407092094421387 seconds
Execution time: 0.07957720756530762 seconds


In [15]:
cv2.destroyAllWindows()
cap.release()

In [2]:
# Import essential libraries 
import requests 
import cv2 
import numpy as np 
import imutils 

# Replace the below URL with your own. Make sure to add "/shot.jpg" at last. 
url = "http://192.168.186.196:8000/stream.mjpg"

# While loop to continuously fetching data from the Url 
while True: 
	img_resp = requests.get(url) 
	print(img_resp)
	# break
	img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8) 
	img = cv2.imdecode(img_arr, -1) 
	img = imutils.resize(img, width=1000, height=1800) 
	cv2.imshow("Android_cam", img) 

	# Press Esc key to exit 
	if cv2.waitKey(1) == 27: 
		break

cv2.destroyAllWindows() 
